In [12]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
from youtube_transcript_api import YouTubeTranscriptApi
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import json

In [14]:
keys = []

In [16]:
def build_youtube_search(developer_key):
    DEVELOPER_KEY = developer_key
    YOUTUBE_API_SERVICE_NAME="youtube"
    YOUTUBE_API_VERSION="v3"
    return build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

def pagination_test(youtube, query):
    search_items = []
    video_items = []
    INIT_REQ = True
    order = "viewCount"
    while len(search_items) < 1000:
        if INIT_REQ:
            search_response = youtube.search().list(
                q = query,
                order = "relevance",
                part = "snippet",
                type = "video",
                videoCaption = "closedCaption",
                maxResults = 50,
                ).execute()
            INIT_REQ = False
        else:
            search_response = youtube.search().list(
                q = query,
                order = "relevance",
                part = "snippet",
                type = "video",
                videoCaption = "closedCaption",
                maxResults = 50,
                pageToken = next_page_token
                ).execute()
            
        search_items.extend(search_response['items'])
        new_results = []
        for s in search_items:
            if s not in new_results:
                new_results.append(s)
        search_items = new_results
        try:
            next_page_token = search_response['nextPageToken']
        except KeyError:
            INIT_REQ = True
            print('Last page. {} items collected.'.format(len(search_items)))
            break
                    
    return search_items, search_response

def my_search(api, query, page_token = ''):
    if not page_token:
        response = api.search().list(
            q = query,
            order = "relevance",
            part = "snippet",
            type = "video",
            videoCaption = "closedCaption",
            maxResults = 50,
            ).execute()
    else:
        response = api.search().list(
            q = query,
            order = "relevance",
            part = "snippet",
            type = "video",
            videoCaption = "closedCaption",
            maxResults = 50,
            pageToken = page_token
            ).execute()
    return response

def get_search_response(youtube, query):
    search_items = []
    video_items = []
    INIT_REQ = True
    prev_ids = []
    i = 0
    while len(search_items) < 1000 and i < 8:
        if INIT_REQ:
            try:
                search_response = my_search(youtube, query)
                INIT_REQ = False
            except HttpError:
                i += 1
                if i >= 8:
                    break
                try:
                    youtube = build_youtube_search(keys[i])
                    search_response = my_search(youtube, query)
                except HttpError:
                    break
        else:
            try:
                search_response = my_search(youtube, query, next_page_token)
            except HttpError:
                i += 1
                if i >= 8:
                    break
                try:
                    youtube = build_youtube_search(keys[i])
                    search_response = my_search(youtube, query, next_page_token)
                except HttpError:
                    break
                    
        search_items.extend(search_response['items'])
        
        new_results = []
        for s in search_items:
            if s not in new_results:
                new_results.append(s)
        search_items = new_results
        
        video_ids = [i.get('id').get('videoId') for i in search_response['items'] if i['id']['kind'] == 'youtube#video']
        if not prev_ids:
            prev_ids = video_ids
            video_ids_new = video_ids
        else:
            video_ids_new = [vid for vid in video_ids if vid not in prev_ids]
            prev_ids += video_ids_new
        
        try:
            video_items.extend(youtube.videos().list(
                part = "id,contentDetails,statistics",
                id = ','.join(video_ids_new)).execute()['items'])
        except HttpError:
            i += 1
            if i >= 8:
                break
            try:
                youtube = build_youtube_search(keys[i])
                video_items.extend(youtube.videos().list(
                    part = "id,contentDetails,statistics",
                    id = ','.join(video_ids_new)).execute()['items'])
            except HttpError:
                break
            
        try:
            next_page_token = search_response['nextPageToken']
        except KeyError:
            INIT_REQ = True
            print('Last page. {} items collected.'.format(len(search_items)))
    
    print('Collecting complete. {} items collected.'.format(len(search_items)))
                    
    return search_items, video_items

def get_video_info(search_items, video_items):
    result_json = {}
    idx = 0
    for item in search_items:
        if item['id']['kind'] == 'youtube#video':
            vid = item['id']['videoId']
            vitems = [item for item in video_items if item['id'] == vid]
            if vitems:
                vitem = vitems[0]
            else:
                continue
            if vitem['contentDetails']['caption']:
                captions = get_captions(item['id']['videoId'])
            if captions:
                result_json[idx] = info_to_dict(item['id']['videoId'], item['snippet']['title'], item['snippet']['description'], vitem['statistics']['viewCount'], captions)
                idx += 1
    return result_json

def info_to_dict(videoId, title, description, viewCount, captions):
    result = {
        "videoId": videoId,
        "title": title,
        "description": description,
        "viewCount": viewCount,
        "captions": captions
    }
    return result

def get_captions(vid):
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(vid)
    except:
        return ''

    for transcript in transcript_list:
        if transcript.language_code == 'en':
            try:
                caption_script = ' '.join([t.get('text') for t in transcript.fetch()])
            except Exception as e:
                print(e)
                return ''
            continue
        if transcript.is_translatable:
            try:
                caption_script = ' '.join([t.get('text') for t in transcript.translate('en').fetch()])
            except Exception as e:
                print(e)
                return ''
        
    return caption_script

In [ ]:
def build_youtube_search(developer_key):
    DEVELOPER_KEY = developer_key
    YOUTUBE_API_SERVICE_NAME="youtube"
    YOUTUBE_API_VERSION="v3"
    return build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

def my_search(api, query, page_token = ''):
    if not page_token:
        response = api.search().list(
            q = query,
            order = "relevance",
            part = "snippet",
            type = "video",
            videoCaption = "closedCaption",
            maxResults = 50,
            ).execute()
    else:
        response = api.search().list(
            q = query,
            order = "relevance",
            part = "snippet",
            type = "video",
            videoCaption = "closedCaption",
            maxResults = 50,
            pageToken = page_token
            ).execute()
    return response

def get_search_response(youtube, query):
    search_items = []
    video_items = []
    INIT_REQ = True
    prev_ids = []
    i = 0
    while len(search_items) < 1000 and i < 8:
        if INIT_REQ:
            try:
                search_response = my_search(youtube, query)
                INIT_REQ = False
            except HttpError:
                i += 1
                if i >= 8:
                    break
                try:
                    youtube = build_youtube_search(keys[i])
                    search_response = my_search(youtube, query)
                except HttpError:
                    break
        else:
            try:
                search_response = my_search(youtube, query, next_page_token)
            except HttpError:
                i += 1
                if i >= 8:
                    break
                try:
                    youtube = build_youtube_search(keys[i])
                    search_response = my_search(youtube, query, next_page_token)
                except HttpError:
                    break
                    
        search_items.extend(search_response['items'])
        
        new_results = []
        for s in search_items:
            if s not in new_results:
                new_results.append(s)
        search_items = new_results
        
        video_ids = [i.get('id').get('videoId') for i in search_response['items'] if i['id']['kind'] == 'youtube#video']
        if not prev_ids:
            prev_ids = video_ids
            video_ids_new = video_ids
        else:
            video_ids_new = [vid for vid in video_ids if vid not in prev_ids]
            prev_ids += video_ids_new
        
        try:
            video_items.extend(youtube.videos().list(
                part = "id,contentDetails,statistics",
                id = ','.join(video_ids_new)).execute()['items'])
        except HttpError:
            i += 1
            if i >= 8:
                break
            try:
                youtube = build_youtube_search(keys[i])
                video_items.extend(youtube.videos().list(
                    part = "id,contentDetails,statistics",
                    id = ','.join(video_ids_new)).execute()['items'])
            except HttpError:
                break
            
        try:
            next_page_token = search_response['nextPageToken']
        except KeyError:
            INIT_REQ = True
            print('Last page. {} items collected.'.format(len(search_items)))
    
    print('Collecting complete. {} items collected.'.format(len(search_items)))
                    
    return search_items, video_items

def get_video_info(search_items, video_items):
    result_json = {}
    idx = 0
    for item in search_items:
        if item['id']['kind'] == 'youtube#video':
            vid = item['id']['videoId']
            vitems = [item for item in video_items if item['id'] == vid]
            if vitems:
                vitem = vitems[0]
            else:
                continue
            if vitem['contentDetails']['caption']:
                captions = get_captions(item['id']['videoId'])
            if captions:
                result_json[idx] = info_to_dict(item['id']['videoId'], item['snippet']['title'], item['snippet']['description'], vitem['statistics']['viewCount'], captions)
                idx += 1
    return result_json

def info_to_dict(videoId, title, description, viewCount, captions):
    result = {
        "videoId": videoId,
        "title": title,
        "description": description,
        "viewCount": viewCount,
        "captions": captions
    }
    return result

def get_captions(vid):
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(vid)
    except:
        return ''

    for transcript in transcript_list:
        if transcript.language_code == 'en':
            try:
                caption_script = ' '.join([t.get('text') for t in transcript.fetch()])
            except Exception as e:
                print(e)
                return ''
            continue
        if transcript.is_translatable:
            try:
                caption_script = ' '.join([t.get('text') for t in transcript.translate('en').fetch()])
            except Exception as e:
                print(e)
                return ''
        
    return caption_script

In [59]:
len(searches)

1005

In [60]:
unique_r = []
for s in searches:
    if s not in unique_r:
        unique_r.append(s)
len(unique_r)

633

In [29]:
youtube_api = build_youtube_search(youtube_api_key)
searches, responses = pagination_test(youtube_api, 'apple iphone 15')

Last page. 588 items collected.


In [37]:
youtube_api = build_youtube_search(keys[0])
searches, details = get_search_response(youtube_api, 'apple iphone 15')

Last page. 537 items collected.
Last page. 605 items collected.
Last page. 623 items collected.
Last page. 635 items collected.
Last page. 637 items collected.
Last page. 638 items collected.
Last page. 642 items collected.
Last page. 647 items collected.
Last page. 648 items collected.
Last page. 650 items collected.
Last page. 654 items collected.
Last page. 654 items collected.
Last page. 655 items collected.
Last page. 655 items collected.
Last page. 655 items collected.
Last page. 656 items collected.
Last page. 659 items collected.
Last page. 660 items collected.
Last page. 662 items collected.
Last page. 664 items collected.
Last page. 664 items collected.
Last page. 666 items collected.
Last page. 666 items collected.
Last page. 666 items collected.
Last page. 666 items collected.
Last page. 668 items collected.
Last page. 671 items collected.
Last page. 677 items collected.
Last page. 677 items collected.
Last page. 678 items collected.
Last page. 679 items collected.
Last pag

In [40]:
videos = get_video_info(searches, details)

In [41]:
videos[0]

{'videoId': 'xqyUdNxWazA',
 'title': 'Introducing iPhone 15 Pro | Apple',
 'description': 'The new iPhone 15 Pro and iPhone 15 Pro Max are ready for action. Built with aerospace-grade titanium for our lightest, most ...',
 'viewCount': '34578451',
 'captions': "[Music] the titanium iPhone 15 Pro built with an aerospace grade material a new level of mobile gaming power and a camera that can literally shoot a blockbuster film wait what a titanium gaming Powerhouse movie camera phone is that even a thing yes it is definitely a thing titanium the same material used in spacecraft design makes iPhone 15 Pro and iPhone 15 Pro Max are lightest Pro Models ever titanium's unique properties give it one of the highest strength to weight ratios of any metal which basically means it's light enough to get up into space and strong enough to make it all the way back to Earth oops the new design comes in four Pro finishes and now includes a USBC connector with super fast transfer speeds and there's a cu

In [42]:
with open('videos_iphone2.json', 'w', encoding='utf-8') as file:
    json.dump(videos, file, indent="\t")
    
with open('videos_iphone2.json', 'r') as file:
    data = json.load(file)

In [45]:
data['0']

{'videoId': 'xqyUdNxWazA',
 'title': 'Introducing iPhone 15 Pro | Apple',
 'description': 'The new iPhone 15 Pro and iPhone 15 Pro Max are ready for action. Built with aerospace-grade titanium for our lightest, most ...',
 'viewCount': '34578451',
 'captions': "[Music] the titanium iPhone 15 Pro built with an aerospace grade material a new level of mobile gaming power and a camera that can literally shoot a blockbuster film wait what a titanium gaming Powerhouse movie camera phone is that even a thing yes it is definitely a thing titanium the same material used in spacecraft design makes iPhone 15 Pro and iPhone 15 Pro Max are lightest Pro Models ever titanium's unique properties give it one of the highest strength to weight ratios of any metal which basically means it's light enough to get up into space and strong enough to make it all the way back to Earth oops the new design comes in four Pro finishes and now includes a USBC connector with super fast transfer speeds and there's a cu

In [18]:
youtube_api = build_youtube_search(keys[0])
searches2, details2 = get_search_response(youtube_api, 'samsung galaxy s23')

Last page. 528 items collected.
Last page. 609 items collected.
Last page. 684 items collected.
Last page. 713 items collected.
Last page. 737 items collected.
Last page. 762 items collected.
Last page. 773 items collected.
Last page. 778 items collected.
Last page. 781 items collected.
Last page. 788 items collected.
Last page. 792 items collected.
Last page. 801 items collected.
Last page. 805 items collected.
Last page. 807 items collected.
Last page. 817 items collected.
Last page. 820 items collected.
Last page. 823 items collected.
Last page. 832 items collected.
Last page. 837 items collected.
Last page. 841 items collected.
Last page. 845 items collected.
Last page. 847 items collected.
Last page. 855 items collected.
Last page. 860 items collected.
Last page. 860 items collected.
Last page. 863 items collected.
Last page. 864 items collected.
Last page. 865 items collected.
Last page. 867 items collected.
Last page. 868 items collected.
Last page. 868 items collected.
Last pag

In [39]:
import pickle
with open('searches.pickle', 'wb') as f:
    pickle.dump(searches, f)
with open('details.pickle', 'wb') as f2:
    pickle.dump(details, f2)

In [20]:
videos2 = get_video_info(searches2, details2)


Could not retrieve a transcript for the video https://www.youtube.com/watch?v=404 Client Error: Not Found for url: https://www.youtube.com/api/timedtext?v=qf98pJzlve4&ei=lUwcZZHnGcGf1d8Pg6OIwAw&caps=asr&opi=112496729&xoaf=5&hl=en&ip=0.0.0.0&ipbits=0&expire=1696378629&sparams=ip,ipbits,expire,v,ei,caps,opi,xoaf&signature=585206292D840758B05E741B6AF20952191085B2.CD213DA95B529F27612B158C95EA89DEE9CC149E&key=yt8&lang=ar&tlang=en! This is most likely caused by:

Request to YouTube failed: qf98pJzlve4

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!


In [21]:
len(videos2)

901

In [22]:
videos2[0]

{'videoId': 'BSYsXVFzmKA',
 'title': 'Galaxy S23 Ultra: Official Introduction Film | Samsung',
 'description': "What's new? The new Galaxy S23 Ultra. Share the epic with our most powerful processor yet, a pro-grade camera that boasts epic ...",
 'viewCount': '24328894',
 'captions': "hi! Nothing bad, right? what's the matter? braided!  Galaxy S23 Ultra. Good. This cannot be expressed simply as ‘good.’ First, the\ndisplay color and contrast are automatically adjusted to match the content you are watching or the surrounding environment. It’s a function called ‘Vision Booster’. Oh really? Now, look! and…  ! Can you drive this? look!  Snapdragon 8 2nd generation for Galaxy! Let's increase GPU, CPU, and NPU. It has the most powerful processor in the Galaxy. Stop, stop, stop! and…  !  I can't stop! This processor is efficient and can keep going. for a moment…  Are we in the water now? fountain!  Concentrate!  The Galaxy S23 Ultra is a product that takes the earth into consideration.  It cons

In [24]:
videos2[100]

{'videoId': 'IKmzHhPb2cI',
 'title': 'Samsung Galaxy S23 Ultra - THIS IS AWESOME',
 'description': 'Samsung Galaxy S23 Ultra will see some significant upgrades in camera, performance, and battery life. But the Galaxy S23 Ultra ...',
 'viewCount': '115156',
 'captions': "all right what's up guys Sal here so we've been talking about how the Galaxy s23 Ultra will offer some pretty significant upgrades in camera performance and even in battery life these three alone make the s23 ultra a substantial upgrade over its predecessor but you see the phone is not just about the camera and performance there are other important elements to it as well that people often Overlook when making a purchasing decision and one of those is the speaker quality most of us have phones in our pockets and use them for almost everything such as watching movies playing games and listening to music while some people may opt to use wireless headphones or Bluetooth speakers to enhance the audio experience the majority 

In [25]:
with open('videos_galaxy2.json', 'w', encoding='utf-8') as file:
    json.dump(videos2, file, indent="\t")
    
with open('videos_galaxy2.json', 'r') as file:
    data = json.load(file)

In [26]:
len(data)

901

In [28]:
type(data)

dict

In [29]:
data['0']

{'videoId': 'BSYsXVFzmKA',
 'title': 'Galaxy S23 Ultra: Official Introduction Film | Samsung',
 'description': "What's new? The new Galaxy S23 Ultra. Share the epic with our most powerful processor yet, a pro-grade camera that boasts epic ...",
 'viewCount': '24328894',
 'captions': "hi! Nothing bad, right? what's the matter? braided!  Galaxy S23 Ultra. Good. This cannot be expressed simply as ‘good.’ First, the\ndisplay color and contrast are automatically adjusted to match the content you are watching or the surrounding environment. It’s a function called ‘Vision Booster’. Oh really? Now, look! and…  ! Can you drive this? look!  Snapdragon 8 2nd generation for Galaxy! Let's increase GPU, CPU, and NPU. It has the most powerful processor in the Galaxy. Stop, stop, stop! and…  !  I can't stop! This processor is efficient and can keep going. for a moment…  Are we in the water now? fountain!  Concentrate!  The Galaxy S23 Ultra is a product that takes the earth into consideration.  It cons

In [101]:
len(videos2)

648

In [88]:
import json
with open('videos.json', 'w', encoding='utf-8') as file:
    json.dump(videos, file, indent="\t")
    
with open('videos.json', 'r') as file:
    data = json.load(file)

In [89]:
len(data)

650

In [31]:
videos = videos2

In [32]:
docs = [v.get('captions') for v in videos.values()]

In [33]:
len(docs)

901

In [34]:
docs = [d for d in docs if d]
len(docs)

901

In [35]:
vectorizer_model = CountVectorizer(stop_words=stopwords.words('english') + ['samsung', 'galaxy', 's23', 'phone'])
topic_model = BERTopic(vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs)

In [36]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12,-1_city_citys_pixel_new,"[city, citys, pixel, new, york, google, milan,...",[welcome to Bree's Odyssey Channel and our cit...
1,0,798,0_ultra_camera_also_like,"[ultra, camera, also, like, one, see, iphone, ...","[Guys, today we're going to do a boxing of the..."
2,1,59,1_tap_screen_want_go,"[tap, screen, want, go, button, see, use, pen,...","[I reviewed the Galaxy S23 Ultra last time, bu..."
3,2,32,2_music_foreign_thank_applause,"[music, foreign, thank, applause, chandra, bos...","[foreign [Music] [Music], foreign [Music] [Mus..."
